### Packages loading

In [ ]:
import numpy as np
import random
import qiskit
import qtm.evolution
import qtm.state
import qtm.qcompilation
import qtm.ansatz
import qtm.constant
from qtm.evolution import environment, mutate, selection, crossover
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2


## 1. Quantum architecture search for quantum state preparation problem

Now, we defind the problem which is need to solve

In [ ]:
n_qubits=4
n=3
theta = qiskit.circuit.ParameterVector('theta',2*n*n_qubits)
#theta = np.random.rand(n*n_qubits)
#print(theta)
qc = qiskit.QuantumCircuit(n_qubits)
for i in range(n_qubits):
    qc.ry(theta[i], i)
for i in range(n_qubits):
    qc.rz(theta[i+n+1], i)
for i in range(n_qubits):
    qc.ry(theta[i+2*(n+1)], i)

for i in range(n):
    qc.cnot(i, i+1)
qc.barrier()

for i in range(n_qubits):
    qc.ry(theta[i+3*(n+1)], i)
for i in range(n_qubits):
    qc.rz(theta[i+4*(n+1)], i)
for i in range(n_qubits):
    qc.ry(theta[i+5*(n+1)], i)

for i in range(n):
    qc.cnot(i, i+1)
qc.draw('mpl')

In [ ]:
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper, QubitConverter, ParityMapper
from qiskit_nature.units import DistanceUnit

driver = PySCFDriver(
    atom="H 0 0 -0.35; H 0 0 0.35",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)
problem = driver.run()
hamiltonian = problem.hamiltonian.second_q_op()

mapper=JordanWignerMapper()
qubit_converter = QubitConverter(mapper)
qubit_op = qubit_converter.convert(hamiltonian)
print(qubit_op)

Refecence values

In [ ]:
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.primitives import Estimator

solver = GroundStateEigensolver(
    mapper,
    NumPyMinimumEigensolver(),
)
result = solver.solve(problem)
print(result)

#estimator = Estimator()
#vqe = VQE(estimator = estimator, ansatz = qc, optimizer=qiskit.algorithms.optimizers.SLSQP(maxiter=100))
#print(f"VQE values: {vqe.compute_minimum_eigenvalue(qubit_op).eigenvalue}")

In [ ]:
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.primitives import Estimator
estimator = Estimator()
def VQE_fitness(qc: qiskit.QuantumCircuit, num_iter = 100):
    vqe = VQE(estimator = estimator, ansatz = qc, optimizer=qiskit.algorithms.optimizers.SLSQP(maxiter=num_iter))
    return vqe.compute_minimum_eigenvalue(qubit_op).eigenvalue.real

Main algorithm, includes: init population, selection, crossover, mutate.

In [ ]:
params = {'depth': 5,
          'num_individual': 8,  # Must mod 8 = 0
          'num_generation': 30,
          'num_qubits': 4,
          'threshold': lambda eigenvalue: np.abs(eigenvalue - (-1.89215)) < 0.001,
          'prob_mutate': 0.01}

env = environment.EEnvironment(
    params,
    fitness_func = VQE_fitness,
    selection_func = selection.elitist_selection,
    crossover_func= crossover.onepoint_crossover,
    mutate_func=mutate.bitflip_mutate,
    pool = qtm.constant.operations
)

env.initialize_population()
env.evol() 

In [ ]:
env.plot()

Test best candidate again

In [ ]:
qc = env.best_candidate.qc
print(VQE_fitness(qc, 100))

In [ ]:
qc.draw('mpl')